그냥 학교에서 교양으로 txt가져와서 단어구름 했던거 가지고 pdf 여러장으로도 만들어본거..
친구 단어구름 필요하다해서 해본거 최적화 X 돌아는감..

wordcloud 설치 오류 날떄 참고:
https://hcid-courses.github.io/TA/FAQ/python_wordcloud_troubleshoot.html

In [1]:
# pip install Pillow numpy pandas PyPDF2 nltk wordcloud konlpy google-colab
# pip install wordcloud --use-pep517
from konlpy.tag import Hannanum
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import konlpy
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from collections import Counter
import nltk
import PyPDF2
import pandas as pd
# from google.colab import drive
# drive.mount('/content/drive')

tokenizer = RegexpTokenizer('[\w]+')

nltk.download('stopwords')
stop_words = stopwords.words('english')

hannanum = Hannanum() 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rldir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# pdf 갯수 입력
nums_pdf = 1  # @개
lines = ""

for i in range(nums_pdf):
    # 한국어 전용 파일이름 krJam0.pdf , krJam1.pdf 무조건 0부터 해야함
    # pdf_file = open(f'/content/drive/My Drive/wordcloud/krJam{i}.pdf', 'rb') #경로 맞게 수정
    # 한국어 전용 파일이름 krjam0.pdf , kerJam1
    # pdf_file = open(f'/content/drive/My Drive/krJam{i}.pdf', 'rb')
    pdf_file = open(f'C:/Users/rldir/OneDrive/바탕 화면/jams/krJam{i}.pdf', 'rb')

    pdf_reader = PyPDF2.PdfReader(pdf_file)
    for page in pdf_reader.pages:
        lines += page.extract_text()

    pdf_file.close()


lines_list = lines.splitlines()
words_list = [lines.split() for line in lines_list]


def flatten(l):
    flatList = []
    for elem in l:
        if type(elem) == list:
            for e in elem:
                flatList.append(e)
        else:
            flatList.append(elem)
    return flatList



In [4]:
double_list = words_list
single_list = [item for sublist in double_list for item in sublist]
lines = single_list
print(len(lines))  # 저번에 한개 햇을떄 683만이었나 그거보다 크면 될듯?


6835590


In [5]:
import multiprocessing

num_cores = multiprocessing.cpu_count()
print(f"총 {num_cores}개의 코어가 감지되었습니다.")


총 8개의 코어가 감지되었습니다.


In [ ]:
# 기존코드

############################# 오래걸림##########주의########
temp = [] 
for i in range(len(lines)):
    print(lines[i]) #줠라김... #되는지 테스트 중단하고 ctrl+/ 하고 다시 실행 ㄱㄱ
    temp.append(hannanum.nouns(lines[i])) #명사만 저장

In [1]:
# gpt로 멀티프로세싱 한 코드
# import multiprocessing as mp

# def get_nouns(line):
#     return hannanum.nouns(line)

# if __name__ == '__main__':
#     temp = [] 
#     pool = mp.Pool(processes=3) # 코어 개수에 맞게 조정
#     for i in range(len(lines)):
#         print(i)
#         #print(lines[i]) #줠라김... #되는지 테스트 중단하고 ctrl+/ 하고 다시 실행 ㄱㄱ
#         temp.append(pool.apply_async(get_nouns, (lines[i],)).get()) #명사만 저장

#     pool.close()
#     pool.join()


In [ ]:
word_list = flatten(temp)
word_list = pd.Series([x for x in word_list if len(x) > 1])
word_list.value_counts().head(30)  # wordcloud에 넣을 빈도 높은 숫자 @개 추출


In [ ]:
font_path = '/content/drive/My Drive/NanumBarunGothic.ttf'

wordcloud = WordCloud(
    font_path=font_path,
    width=800,  # 가로크기 #저장하는거에 영향 미침ㅇㅇ
    height=800,  # 세로크기
    background_color="white"  # 뒷 배경
)

count = Counter(word_list)
wordcloud = wordcloud.generate_from_frequencies(count)


def __array__(self):
    """Convert to numpy array.
    Returns
    -------
    image : nd-array size (width, height, 3)
        Word cloud image as numpy matrix.
    """
    return self.to_array()


def to_array(self):
    """Convert to numpy array.
    Returns
    -------
    image : nd-array size (width, height, 3)
        Word cloud image as numpy matrix.
    """
    return np.array(self.to_image())


array = wordcloud.to_array()
print(count)  # 클라우드에 들어갈 것들 이상한거 보고 제거하기 (출력 윗 줄)
# 밑에 불필요한거 입력
del_items = ""  # 불필요한거 입력 ㄱㄱ

del_items = del_items.split()

for del_item in del_items:
    del count[del_item]
###############################
print(count)  # 제거된 이후 결과 실행하면 나오는 줄중에 (밑에 줄)

wordcloud = wordcloud.generate_from_frequencies(count)
array = wordcloud.to_array()


In [ ]:
if __name__ == '__main__':  # 대체 코드
    plt.show()


# 얼마나 크게 보여주냐 #저장하는거에 영향 X 밑에 보여주기 크기용 #아마도?
fig = plt.figure(figsize=(10, 10))
plt.imshow(array, interpolation="bilinear")
plt.show()  # 보여주기
fig.savefig('wordcloud.png')  # 파일이 저장되는 이름 ㅇㅇ
